In [2]:
import snowflake.snowpark
from snowflake.snowpark import functions as F
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit,count
from snowflake.snowpark.version import VERSION
# Misc
import json
import pandas as pd
import numpy as np
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

In [3]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION

# # Current Environment Details
# print('User                        : {}'.format(snowflake_environment[0][0]))
# print('Role                        : {}'.format(snowflake_environment[0][1]))
# print('Database                    : {}'.format(snowflake_environment[0][2]))
# print('Schema                      : {}'.format(snowflake_environment[0][3]))
# print('Warehouse                   : {}'.format(snowflake_environment[0][5]))
# print('Snowflake version           : {}'.format(snowflake_environment[0][4]))
# print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

In [4]:
ca_zip = json.load(open('src/zip_json.json'))

In [6]:
%%writefile pages/CLV_prediction.py
import pandas as pd
import streamlit as st
import json
import numpy as np
import sys
import cachetools
import joblib
# from streamlit.report_thread import get_report_ctx
# from streamlit.server.server import Server
import snowflake.snowpark
from snowflake.snowpark import functions as F
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import PandasDataFrame,PandasSeries
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit,count
from snowflake.snowpark.version import VERSION

st.set_page_config(page_title= "Customer Lifetime Value")

st.image(
    "src/1649251328-maximize-your-clv.webp",
    width = 600,
)

#st.title("XGBoost model to predict CLV")

col1, col2 = st.columns(2,gap = "medium")
with col1:
    st.markdown('#### Numeric Features')
    Cus_by = st.number_input('Customer Birth Year:', 
                    min_value=1924,
                    max_value=2020, 
                    help = 'Please type VALID birth Year!!(Range: 1924~2020)'
                            )

    Cs_zip = st.number_input( 'Customer Zip Code:', 
                    min_value= 601, 
                    max_value= 99981,
                    value  = 66668,
                    step = 1
                            )


with col2:
    st.markdown('#### Categorical Features ')
    Cus_gender = st.selectbox('CD_Gender',
                              ['M', 'F'], 
                              help= 'M: Male, F: Female'
     )

    Cus_marital = st.selectbox( 'CD_MARITAL_STATUS',
                 ['S','D','W', 'U', 'M'],
     )
    Cus_credit = st.selectbox('CD_CREDIT_RATING',
                 ['Low Risk','Unknown','Good','High Risk']
       )

    Cus_edu = st.selectbox( 'CD_EDUCATION_STATUS',
                 ['Advanced Degree','Secondary','2 yr Degree','4 yr Degree','Unknown','Primary','College']
                )
    Cus_dep = st.selectbox( 'CD_DEP_COUNT',
                 ['0', '1'],
      )



col3, col4 = st.columns([8,2],gap = "medium")

with col3:
    model_select = st.radio('Select the Model here:',
                           [ 'XGBoost','Linear Regression']
                            #[ 'XGBoost']
                           )
with col4:
    submit =  st.button('Submit')
    reset = st.button('Reset ')
@st.cache_resource
def initialize_SF():
    connection_parameters = json.load(open('connection.json'))
    session = Session.builder.configs(connection_parameters).create()
    session.sql_simplifier_enabled = True
    ca_zip = json.load(open('src/zip_json.json'))    
    # set up feature engineering/inference warehouse
    session.use_warehouse('FE_AND_INFERENCE_WH')
    session.use_database('tpcds_xgboost')
    session.use_schema('demo')
    session.add_packages('snowflake-snowpark-python', 'scikit-learn', 'pandas', 'numpy', 'joblib', 'cachetools', 'xgboost==1.5.0', 'joblib')
    session.add_import("@ml_models_10T/model.joblib")  
    session.add_import("@ml_models_LR_10T/model_LR.joblib")
    return session, ca_zip
session, ca_zip = initialize_SF()

# choose model here
if model_select == 'XGBoost':    
    stage_name = 'ml_models_10T'
    model_name = 'model.joblib'
    
else:
    stage_name = 'ml_models_LR_10T'
    model_name = 'model_LR.joblib'

features = [ 'C_BIRTH_YEAR', 'CA_ZIP', 'CD_GENDER', 'CD_MARITAL_STATUS', 'CD_CREDIT_RATING', 'CD_EDUCATION_STATUS', 'CD_DEP_COUNT']

@cachetools.cached(cache={})
def read_file(filename):
    import os, joblib
    import_dir = sys._xoptions.get("snowflake_import_directory")
    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = joblib.load(file)
            return m

@F.pandas_udf(session=session, max_batch_size=10000, is_permanent=True, stage_location=f'@{stage_name}', replace=True, name="clv_xgboost_udf")
def predict(df:  PandasDataFrame[int, str, str, str, str, str, int]) -> PandasSeries[float]:
    m = read_file(model_name)       
    df.columns = features
    return m.predict(df) 

# if click submit
if submit:
    typed_input = [[Cus_by, ca_zip.get(str(Cs_zip), '66668'),Cus_gender,Cus_marital,Cus_credit,Cus_edu, int(Cus_dep)]]
    #st.write(typed_input)
    input_df = session.create_dataframe(typed_input, schema=features)
    typed_output = input_df.select(*input_df,
                    predict(*input_df).alias('PREDICTION'))
    output = pd.DataFrame(typed_output.collect()).T
    output.columns = ['']
    st.write(output)
# if click reset  
if reset:
    st.write('放弃reset？')
    

    


Overwriting pages/CLV_prediction.py


In [20]:
%%writefile Customer_Lifetime_Value.py

import pandas as pd
import numpy as np
import streamlit as st
import json
from sqlalchemy import create_engine
st.set_page_config(page_title= "Sqlalchemy Query")

# initialize the engine and read chached data from json files
@st.cache_resource
def initialize():
    engine = create_engine(
        'snowflake://{user}:{password}@{account}/d'.format(
            user='ESTPEGION',
            password='SnowFlake1234!',
            account='mg61873.ca-central-1.aws',
            database = 'SNOWFLAKE_SAMPLE_DATA',
            schema = 'TPCDS_SF10TCL',
            warehouse = 'COMPUTE_WH',
            role='accountadmin',
            numpy = True
        )
    )
    
    connection = engine.connect()
    # pd.read_sql_query('''USE SCHEMA SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL;''',engine)
    var_json = json.load(open('var_store.json'))
    return engine, var_json

engine, var_json = initialize()
pd.read_sql_query('''USE SCHEMA SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL;''',engine)
st.image(
    "src/data.jpeg",
    caption='Query the Data',
     use_column_width  = 'always',
)

# input GUI for user
# _,col1,_ = st.columns([1,8,1])
# _,col2,_ = st.columns([1,8,1])
col1, col2 = st.columns(2,gap = "medium")

with col1:
    # lai's contribution here
    Query_selection = st.selectbox('Select the Query here:',
                            [ 'Q1','Q2','Q3','Q4','Q5','Q6','Q43','Q60']
                           )
    st.markdown('#### Features')
    if Query_selection == 'Q1':
        
        form = st.form(key='Q1-form')
        year_input = form.number_input('year',
                                        min_value=1900,
                                        max_value=2100,
                                        value  = 2000,
                                        help = 'Input value not in range.(Range: 1900~2100)')
        state_input = form.selectbox('State', 
                                     var_json['q1_state']
                                     )
        
        agg_input = form.selectbox('Aggreagation Column',var_json['q1_agg'])

        submit = form.form_submit_button('Submit')

    elif Query_selection == 'Q2':
        form = st.form(key='Q2-form')
        year_input = form.number_input('year',
                                        min_value=1900,
                                        max_value=2100,
                                        value  = 2000,
                                        help = 'Input value not in range.(Range: 1900~2100)')
        submit = form.form_submit_button('Submit')
        
        
    elif Query_selection == 'Q3':
        '''
        Q3: Report the total extended sales price per item brand of a specific manufacturer for all sales in a specific month
        of the year.
        '''
        form = st.form(key='Q3-form')
        month_input = form.number_input('Month',
                                        min_value=1,
                                        max_value=12,
                                        value = 11,
                                        help = 'Input value should in this range.(Range: 1~12)')
        
        MANUFACT_input = form.number_input('MANUFACT',
                                        min_value=12,
                                        max_value=998,
                                        value = 128,
                                        help = 'Input value should in this range.(Range: 12~998)')
        
        aggc_input = form.selectbox('AGGC',var_json['q3_aggc'])                
        
        submit = form.form_submit_button('Submit')
        
        
    elif Query_selection == 'Q4':
        form = st.form(key='Q4-form')
        year_input = form.number_input('year',
                                        min_value=1900,
                                        max_value=2100,
                                        value  = 2000,
                                        help = 'Input value not in range.(Range: 1900~2100)')
        agg_input = form.selectbox('Aggregation Column',
                                  var_json['q4_agg'])
        
        submit = form.form_submit_button('Submit')
        
        pass
        
    elif Query_selection == 'Q5':
        pass
        
    elif Query_selection == 'Q40':
        pass
        
    elif Query_selection == 'Q43':
        pass
        
    elif Query_selection == 'Q60':
        
        '''
        Q60: What is the monthly sales amount for a specific month in a specific year, for items in a specific category,
        purchased by customers residing in a specific time zone. Group sales by item and sort output by sales amount.
        '''
        form = st.form(key='Q60-form')

        month_input = form.number_input('Month',
                                            min_value=1,
                                            max_value=12,
                                            value = 9,
                                            help = 'Input value should in this range.(Range: 1~12)')

        year_input = form.number_input('year',
                                            min_value=1900,
                                            max_value=2100,
                                            value  = 1998,
                                            help = 'Input value not in range.(Range: 1900~2100)')

        category_input = form.selectbox('I_Category',["Music","Sports","Children","Women",'None',
                                                              "Home","Electronics","Jewelry","Men","Shoes","Books"])
        submit = form.form_submit_button('Submit')
        
    else:
        st.markdown('Please Select your Query')
    st.write('Press submit run the query for chosen question.')

# outputs if click submit
with col2:
    if Query_selection == 'Q1' and submit:
        q1_state = state_input
        q1_year = year_input
        q1_agg = agg_input
        q1 = '''with customer_total_return as
                (select sr_customer_sk as ctr_customer_sk
                ,sr_store_sk as ctr_store_sk
                ,sum({agg}) as ctr_total_return
                from store_returns
                ,date_dim
                where sr_returned_date_sk = d_date_sk
                and d_year = {year}
                group by sr_customer_sk
                ,sr_store_sk
                limit 500)
                 select  c_customer_id
                from customer_total_return ctr1
                ,store
                ,customer
                where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
                from customer_total_return ctr2
                where ctr1.ctr_store_sk = ctr2.ctr_store_sk)
                and s_store_sk = ctr1.ctr_store_sk
                and s_state = \'{state}\'
                and ctr1.ctr_customer_sk = c_customer_sk
                order by c_customer_id
                 limit 100;'''.format(year = q1_year, state = q1_state, agg = q1_agg)
        
        st.write(pd.read_sql_query(q1 ,engine))
    
    elif Query_selection == 'Q2' and submit:
        # the query runs super slow
        st.write(pd.read_sql_query('select * from WEB_SALES limit 10;',engine))
        q2 = f'''WITH wscs AS
  (SELECT sold_date_sk,
          sales_price
   FROM
     (SELECT ws_sold_date_sk sold_date_sk,
             ws_ext_sales_price sales_price
      FROM web_sales
      UNION ALL SELECT cs_sold_date_sk sold_date_sk,
                       cs_ext_sales_price sales_price
      FROM catalog_sales) sq1),
     wswscs AS
  (SELECT d_week_seq,
          sum(CASE
                  WHEN (d_day_name='Sunday') THEN sales_price
                  ELSE NULL
              END) sun_sales,
          sum(CASE
                  WHEN (d_day_name='Monday') THEN sales_price
                  ELSE NULL
              END) mon_sales,
          sum(CASE
                  WHEN (d_day_name='Tuesday') THEN sales_price
                  ELSE NULL
              END) tue_sales,
          sum(CASE
                  WHEN (d_day_name='Wednesday') THEN sales_price
                  ELSE NULL
              END) wed_sales,
          sum(CASE
                  WHEN (d_day_name='Thursday') THEN sales_price
                  ELSE NULL
              END) thu_sales,
          sum(CASE
                  WHEN (d_day_name='Friday') THEN sales_price
                  ELSE NULL
              END) fri_sales,
          sum(CASE
                  WHEN (d_day_name='Saturday') THEN sales_price
                  ELSE NULL
              END) sat_sales
   FROM wscs,
        date_dim
   WHERE d_date_sk = sold_date_sk
   GROUP BY d_week_seq
   limit 1000)
SELECT d_week_seq1,
       round(sun_sales1/sun_sales2, 2),
       round(mon_sales1/mon_sales2, 2),
       round(tue_sales1/tue_sales2, 2),
       round(wed_sales1/wed_sales2, 2),
       round(thu_sales1/thu_sales2, 2),
       round(fri_sales1/fri_sales2, 2),
       round(sat_sales1/sat_sales2, 2)
FROM
  (SELECT wswscs.d_week_seq d_week_seq1,
          sun_sales sun_sales1,
          mon_sales mon_sales1,
          tue_sales tue_sales1,
          wed_sales wed_sales1,
          thu_sales thu_sales1,
          fri_sales fri_sales1,
          sat_sales sat_sales1
   FROM wswscs,
        date_dim
   WHERE date_dim.d_week_seq = wswscs.d_week_seq
     AND d_year = {year_input}) y,
  (SELECT wswscs.d_week_seq d_week_seq2,
          sun_sales sun_sales2,
          mon_sales mon_sales2,
          tue_sales tue_sales2,
          wed_sales wed_sales2,
          thu_sales thu_sales2,
          fri_sales fri_sales2,
          sat_sales sat_sales2
   FROM wswscs,
        date_dim
   WHERE date_dim.d_week_seq = wswscs.d_week_seq
     AND d_year = {year_input}+1) z
WHERE d_week_seq1 = d_week_seq2-53
ORDER BY d_week_seq1
limit 1000;
'''
        st.write(pd.read_sql_query(q2 ,engine))
        
    elif Query_selection == 'Q3' and submit:
#         st.write(pd.read_sql_query('select * from WEB_SALES limit 10;',engine))
        q3 = f'''
        SELECT dt.d_year,
       item.i_brand_id brand_id,
       item.i_brand brand,
       sum(ss_ext_sales_price) sum_agg
FROM date_dim dt,
     store_sales,
     item
WHERE dt.d_date_sk = store_sales.ss_sold_date_sk
  AND store_sales.ss_item_sk = {aggc_input}
  AND item.i_manufact_id = {MANUFACT_input}
  AND dt.d_moy={month_input}
GROUP BY dt.d_year,
         item.i_brand,
         item.i_brand_id
ORDER BY dt.d_year,
         sum_agg DESC,
         brand_id
LIMIT 100;'''
        st.write(pd.read_sql_query(q3,engine))
        
    elif Query_selection == 'Q4' and submit:
        agg_input = 'c_'+agg_input
        alis_input = 'customer_'+agg_input
        
        q4 = '''WITH year_total AS
  (SELECT c_customer_id customer_id,
          c_first_name customer_first_name,
          c_last_name customer_last_name,
          c_preferred_cust_flag customer_preferred_cust_flag,
          c_birth_country customer_birth_country,
          c_login customer_login,
          c_email_address customer_email_address,
          d_year dyear,
          sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total,
          's' sale_type
   FROM customer,
        store_sales,
        date_dim
   WHERE c_customer_sk = ss_customer_sk
     AND ss_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
   UNION ALL SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((cs_ext_list_price-cs_ext_wholesale_cost-cs_ext_discount_amt)+cs_ext_sales_price)/2)) year_total,
                    'c' sale_type
   FROM customer,
        catalog_sales,
        date_dim
   WHERE c_customer_sk = cs_bill_customer_sk
     AND cs_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
   UNION ALL SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((ws_ext_list_price-ws_ext_wholesale_cost-ws_ext_discount_amt)+ws_ext_sales_price)/2)) year_total,
                    'w' sale_type
   FROM customer,
        web_sales,
        date_dim
   WHERE c_customer_sk = ws_bill_customer_sk
     AND ws_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year)
SELECT t_s_secyear.customer_id,
       t_s_secyear.customer_first_name,
       t_s_secyear.customer_last_name,
       t_s_secyear.customer_preferred_cust_flag
FROM year_total t_s_firstyear,
     year_total t_s_secyear,
     year_total t_c_firstyear,
     year_total t_c_secyear,
     year_total t_w_firstyear,
     year_total t_w_secyear
WHERE t_s_secyear.customer_id = t_s_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_c_secyear.customer_id
  AND t_s_firstyear.customer_id = t_c_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_secyear.customer_id
  AND t_s_firstyear.sale_type = 's'
  AND t_c_firstyear.sale_type = 'c'
  AND t_w_firstyear.sale_type = 'w'
  AND t_s_secyear.sale_type = 's'
  AND t_c_secyear.sale_type = 'c'
  AND t_w_secyear.sale_type = 'w'
  AND t_s_firstyear.dyear = 2001
  AND t_s_secyear.dyear = 2001+1
  AND t_c_firstyear.dyear = 2001
  AND t_c_secyear.dyear = 2001+1
  AND t_w_firstyear.dyear = 2001
  AND t_w_secyear.dyear = 2001+1
  AND t_s_firstyear.year_total > 0
  AND t_c_firstyear.year_total > 0
  AND t_w_firstyear.year_total > 0
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_s_firstyear.year_total > 0 THEN t_s_secyear.year_total / t_s_firstyear.year_total
                ELSE NULL
            END
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_w_firstyear.year_total > 0 THEN t_w_secyear.year_total / t_w_firstyear.year_total
                ELSE NULL
            END
ORDER BY t_s_secyear.customer_id,
         t_s_secyear.customer_first_name,
         t_s_secyear.customer_last_name,
         t_s_secyear.customer_preferred_cust_flag
LIMIT 100;'''
        q4 = q4.replace('c_preferred_cust_flag', agg_input).replace('customer_preferred_cust_flag', alis_input).replace('2001', str(year_input))
        pd.read_sql_query(q4, engine)
        
        pass
    elif Query_selection == 'Q5' and submit:
        pass
    elif Query_selection == 'Q6' and submit:
        pass
    elif Query_selection == 'Q43' and submit:
        pass
    elif Query_selection == 'Q60' and submit:
        q60 = f'''
                WITH ss AS
        (SELECT i_item_id,
                sum(ss_ext_sales_price) total_sales
        FROM store_sales,
                date_dim,
                customer_address,
                item
        WHERE i_item_id IN
            (SELECT i_item_id
                FROM item
                WHERE i_category ={category_input})
            AND ss_item_sk = i_item_sk
            AND ss_sold_date_sk = d_date_sk
            AND d_year = {year_input}
            AND d_moy = {month_input}
            AND ss_addr_sk = ca_address_sk
            AND ca_gmt_offset = -5
        GROUP BY i_item_id limit 1000) limit 1000,
            cs AS
        (SELECT i_item_id,
                sum(cs_ext_sales_price) total_sales
        FROM catalog_sales,
                date_dim,
                customer_address,
                item
        WHERE i_item_id IN
            (SELECT i_item_id
                FROM item
                WHERE i_category ={category_input} limit 1000)
            AND cs_item_sk = i_item_sk
            AND cs_sold_date_sk = d_date_sk
            AND d_year = {year_input}
            AND d_moy = {month_input}
            AND cs_bill_addr_sk = ca_address_sk
            AND ca_gmt_offset = -5
        GROUP BY i_item_id ) limit 1000,
            ws AS
        (SELECT i_item_id,
                sum(ws_ext_sales_price) total_sales
        FROM web_sales,
                date_dim,
                customer_address,
                item
        WHERE i_item_id IN
            (SELECT i_item_id
                FROM item
                WHERE i_category = {category_input} limit 1000)
            AND ws_item_sk = i_item_sk
            AND ws_sold_date_sk = d_date_sk
            AND d_year = {year_input}
            AND d_moy = {month_input}
            AND ws_bill_addr_sk = ca_address_sk
            AND ca_gmt_offset = -5
        GROUP BY i_item_id
        limit 1000)
        
        SELECT i_item_id,
            sum(total_sales) total_sales
        FROM
        (SELECT *
        FROM ss
        UNION ALL SELECT *
        FROM cs
        UNION ALL SELECT *
        FROM ws limit 1000) tmp1
        GROUP BY i_item_id
        ORDER BY i_item_id,
                total_sales
        LIMIT 100;        
        '''
        st.write(pd.read_sql_query(q60,engine))
    elif submit:
        st.write('Wrong Question Number.')

    



Overwriting Customer_Lifetime_Value.py


In [ ]:
#q4 test

agg_input = 'c_'+agg_input
alis_input = 'customer_'+agg_input

pd.read_sql_query('''WITH year_total AS
  (SELECT c_customer_id customer_id,
          c_first_name customer_first_name,
          c_last_name customer_last_name,
          {agg} {alis},
          c_birth_country customer_birth_country,
          c_login customer_login,
          c_email_address customer_email_address,
          d_year dyear,
          sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total,
          's' sale_type
   FROM customer,
        store_sales,
        date_dim
   WHERE c_customer_sk = ss_customer_sk
     AND ss_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
   UNION ALL SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((cs_ext_list_price-cs_ext_wholesale_cost-cs_ext_discount_amt)+cs_ext_sales_price)/2)) year_total,
                    'c' sale_type
   FROM customer,
        catalog_sales,
        date_dim
   WHERE c_customer_sk = cs_bill_customer_sk
     AND cs_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
   UNION ALL SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((ws_ext_list_price-ws_ext_wholesale_cost-ws_ext_discount_amt)+ws_ext_sales_price)/2)) year_total,
                    'w' sale_type
   FROM customer,
        web_sales,
        date_dim
   WHERE c_customer_sk = ws_bill_customer_sk
     AND ws_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year)
SELECT t_s_secyear.customer_id,
       t_s_secyear.customer_first_name,
       t_s_secyear.customer_last_name,
       t_s_secyear.customer_preferred_cust_flag
FROM year_total t_s_firstyear,
     year_total t_s_secyear,
     year_total t_c_firstyear,
     year_total t_c_secyear,
     year_total t_w_firstyear,
     year_total t_w_secyear
WHERE t_s_secyear.customer_id = t_s_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_c_secyear.customer_id
  AND t_s_firstyear.customer_id = t_c_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_secyear.customer_id
  AND t_s_firstyear.sale_type = 's'
  AND t_c_firstyear.sale_type = 'c'
  AND t_w_firstyear.sale_type = 'w'
  AND t_s_secyear.sale_type = 's'
  AND t_c_secyear.sale_type = 'c'
  AND t_w_secyear.sale_type = 'w'
  AND t_s_firstyear.dyear = {year}
  AND t_s_secyear.dyear = {year}+1
  AND t_c_firstyear.dyear = {year}
  AND t_c_secyear.dyear = {year}+1
  AND t_w_firstyear.dyear = {year}
  AND t_w_secyear.dyear = {year}+1
  AND t_s_firstyear.year_total > 0
  AND t_c_firstyear.year_total > 0
  AND t_w_firstyear.year_total > 0
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_s_firstyear.year_total > 0 THEN t_s_secyear.year_total / t_s_firstyear.year_total
                ELSE NULL
            END
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_w_firstyear.year_total > 0 THEN t_w_secyear.year_total / t_w_firstyear.year_total
                ELSE NULL
            END
ORDER BY t_s_secyear.customer_id,
         t_s_secyear.customer_first_name,
         t_s_secyear.customer_last_name,
         t_s_secyear.customer_preferred_cust_flag
LIMIT 100;''').format(year = year_input, agg = agg_input, alis = alis_input)

In [18]:
q4 = '''WITH year_total AS
  (SELECT c_customer_id customer_id,
          c_first_name customer_first_name,
          c_last_name customer_last_name,
          c_preferred_cust_flag customer_preferred_cust_flag,
          c_birth_country customer_birth_country,
          c_login customer_login,
          c_email_address customer_email_address,
          d_year dyear,
          sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total,
          's' sale_type
   FROM customer,
        store_sales,
        date_dim
   WHERE c_customer_sk = ss_customer_sk
     AND ss_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
   UNION ALL SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((cs_ext_list_price-cs_ext_wholesale_cost-cs_ext_discount_amt)+cs_ext_sales_price)/2)) year_total,
                    'c' sale_type
   FROM customer,
        catalog_sales,
        date_dim
   WHERE c_customer_sk = cs_bill_customer_sk
     AND cs_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
   UNION ALL SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((ws_ext_list_price-ws_ext_wholesale_cost-ws_ext_discount_amt)+ws_ext_sales_price)/2)) year_total,
                    'w' sale_type
   FROM customer,
        web_sales,
        date_dim
   WHERE c_customer_sk = ws_bill_customer_sk
     AND ws_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year)
SELECT t_s_secyear.customer_id,
       t_s_secyear.customer_first_name,
       t_s_secyear.customer_last_name,
       t_s_secyear.customer_preferred_cust_flag
FROM year_total t_s_firstyear,
     year_total t_s_secyear,
     year_total t_c_firstyear,
     year_total t_c_secyear,
     year_total t_w_firstyear,
     year_total t_w_secyear
WHERE t_s_secyear.customer_id = t_s_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_c_secyear.customer_id
  AND t_s_firstyear.customer_id = t_c_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_secyear.customer_id
  AND t_s_firstyear.sale_type = 's'
  AND t_c_firstyear.sale_type = 'c'
  AND t_w_firstyear.sale_type = 'w'
  AND t_s_secyear.sale_type = 's'
  AND t_c_secyear.sale_type = 'c'
  AND t_w_secyear.sale_type = 'w'
  AND t_s_firstyear.dyear = 2001
  AND t_s_secyear.dyear = 2001+1
  AND t_c_firstyear.dyear = 2001
  AND t_c_secyear.dyear = 2001+1
  AND t_w_firstyear.dyear = 2001
  AND t_w_secyear.dyear = 2001+1
  AND t_s_firstyear.year_total > 0
  AND t_c_firstyear.year_total > 0
  AND t_w_firstyear.year_total > 0
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_s_firstyear.year_total > 0 THEN t_s_secyear.year_total / t_s_firstyear.year_total
                ELSE NULL
            END
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_w_firstyear.year_total > 0 THEN t_w_secyear.year_total / t_w_firstyear.year_total
                ELSE NULL
            END
ORDER BY t_s_secyear.customer_id,
         t_s_secyear.customer_first_name,
         t_s_secyear.customer_last_name,
         t_s_secyear.customer_preferred_cust_flag
LIMIT 100;'''
q4 = q4.replace('c_preferred_cust_flag', agg_input).replace('customer_preferred_cust_flag', alis_input).replace('2001', year_input)

NameError: name 'agg_input' is not defined

In [17]:
# q4 = q4.replace('c_preferred_cust_flag', agg_input).replace('customer_preferred_cust_flag', alis_input).replace('2001', year_input)
print(q4)

WITH year_total AS
  (SELECT c_customer_id customer_id,
          c_first_name customer_first_name,
          c_last_name customer_last_name,
          c_preferred_cust_flag customer_preferred_cust_flag,
          c_birth_country customer_birth_country,
          c_login customer_login,
          c_email_address customer_email_address,
          d_year dyear,
          sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total,
          's' sale_type
   FROM customer,
        store_sales,
        date_dim
   WHERE c_customer_sk = ss_customer_sk
     AND ss_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
   UNION ALL SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
     

In [11]:
import json
var_json = json.load(open('var_store.json'))
var_json['q1_agg']

['SR_RETURN_AMT',
 'SR_RETURN_AMT_INC_TAX',
 'SR_RETURN_QUANTITY',
 'SR_RETURN_TAX',
 'SR_RETURN_SHIP_COST']